# GPU Information

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
# memory footprint support libraries/code

!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]

def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm() 

In [ ]:
#!kill -9 -1

# Data Warehousing Setup



In [ ]:
# Mount Google Drive folder

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Add models repos and save space 

%cd  /content/drive/My Drive/Models Running/Summarization

# Lex Rank Model


In [ ]:
! pip install lexrank

In [ ]:
#! rm -rf lexrank

In [ ]:
! git clone https://github.com/crabcamp/lexrank.git

In [ ]:
% cd /content/drive/My Drive/Models Running/Summarization/lexrank

import os
import pandas as pd

from lexrank import STOPWORDS, LexRank
from path import Path

import warnings
warnings.filterwarnings("ignore")

genre = pd.read_csv('/content/drive/My Drive/Models Running/Summarization/datasets/genre_final_for_summarization.tsv', sep='\t')

In [ ]:
documents = []
documents_dir = Path('/content/drive/My Drive/Models Running/Summarization/datasets/training/lexrank/bbc')

folders = [dI for dI in os.listdir(documents_dir) if os.path.isdir(os.path.join(documents_dir,dI))]

for folder_name in folders:
  if(folder_name != 'sport'):
    subdirectory = documents_dir + '/' + folder_name
    print(subdirectory)
  
  for file_path in subdirectory.files('*.txt'):
      with file_path.open(mode='rt', encoding='utf-8') as fp:
          documents.append(fp.readlines())

print('LexRank...')
lxr = LexRank(documents, stopwords=STOPWORDS['en'])

In [ ]:
genre.drop(columns=['song', 'artist', 'genre', 'lyrics'], inplace=True)
genre.set_index('genre_encoded', inplace=True)

In [ ]:
def create_summary(line):
  
  # get summary with continuous LexRank
  summary = lxr.get_summary(line.values[0].split('\n'), threshold=None)
  
  return summary[0]

In [ ]:
print("Starting with Genre...\n\n")
genre_textrank = genre.apply( lambda line: create_summary(line), axis=1) 

In [ ]:
genre_textrank = genre_textrank[genre_textrank.str.len() >= 25]

genre_textrank.to_csv('/content/drive/My Drive/Models Running/Summarization/datasets/lex_rank/genre.tsv', sep='\t', header=False, index_label=False)